In [9]:
import xgboost as xgb

import pandas as pd
import pickle
import numpy as np
import math
import pymongo

%pylab inline
pd.set_option('display.mpl_style', 'default')
figsize(12, 9)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from pymongo import MongoClient, DESCENDING

# from pymongo.objectid import ObjectId
import bson

import os
import json

Populating the interactive namespace from numpy and matplotlib


In [10]:
from bson.objectid import ObjectId

In [11]:
client = MongoClient()
db = client.dataframe

In [12]:
new_embds = []
for e in db.short_embeddings.find({}):
    for vs in e.values():
        if isinstance(vs, ObjectId):
            continue
        else:
            ev = vs
    
    new_embds.append(ev)

In [13]:
new_embds = np.array(new_embds)
new_embds.shape

(122523, 307)

In [14]:
columns = range(new_embds.shape[1])
columns[6] = 'sex'
columns[0] = 'user_id'

In [15]:
df = pd.DataFrame(data=new_embds, columns=columns)

In [16]:
df.shape

(122523, 307)

In [17]:
from in_relation import make_model_upd

In [20]:
params = {
    'bst:max_depth':3,
    'bst:eta':1,
    'gamma': 1.,
    'scale_pos_weight':0.7,
    'lambda': 1.,
    'alpha':2.,
    'booster': 'dart',
    'objective':'binary:logistic' 
}

In [21]:
%%time
make_model_upd(df, fname='kekek.model', params=params)

[0]	eval-auc:0.639751	train-auc:0.655569
[1]	eval-auc:0.684471	train-auc:0.692261
[2]	eval-auc:0.684578	train-auc:0.692997
[3]	eval-auc:0.695908	train-auc:0.704494
[4]	eval-auc:0.698224	train-auc:0.707964
[5]	eval-auc:0.698756	train-auc:0.709037
[6]	eval-auc:0.699305	train-auc:0.709527
[7]	eval-auc:0.705582	train-auc:0.714775
[8]	eval-auc:0.705895	train-auc:0.7151
[9]	eval-auc:0.709214	train-auc:0.720744
[10]	eval-auc:0.713582	train-auc:0.7249
[11]	eval-auc:0.715188	train-auc:0.726241
[12]	eval-auc:0.714932	train-auc:0.727551
[13]	eval-auc:0.714526	train-auc:0.72723
[14]	eval-auc:0.716588	train-auc:0.728616
[15]	eval-auc:0.7186	train-auc:0.730454
[16]	eval-auc:0.718164	train-auc:0.73068
[17]	eval-auc:0.720054	train-auc:0.733888
[18]	eval-auc:0.721041	train-auc:0.735339
[19]	eval-auc:0.721139	train-auc:0.736524
[20]	eval-auc:0.721026	train-auc:0.736699
[21]	eval-auc:0.720819	train-auc:0.736911
[22]	eval-auc:0.720978	train-auc:0.737099
[23]	eval-auc:0.721464	train-auc:0.738496
[24]	eval-

In [22]:
# xgb.plot_importance(bst)

In [23]:
from in_relation import make_prediction

In [24]:
db_users_and_features = client['users_and_features']
posts = db_users_and_features.posts

In [25]:

embd = posts.find_one({'active_search_score': 1})['profile_embedding']
del embd[6]
# columns = [str(e) for e in range(len(embd))]
# columns[6] = 'sex'
# test = pd.DataFrame([embd], columns=columns)

In [28]:
# test.shape

In [30]:
# df_copy = test.copy()

bst_loaded = xgb.Booster({'nthread': 24}) # init model
bst_loaded.load_model('kekek.model') # load data
    
# if 'sex' in df_copy:
#     del df_copy['sex']

test_xgb = xgb.DMatrix([embd])
bst_loaded.predict(test_xgb)[0]


0.28959474

In [31]:
# test = make_prediction(test, fname='kekek.model')

In [32]:
bst_loaded = xgb.Booster({'nthread': 24}) # init model
bst_loaded.load_model('kekek.model') # load data

In [35]:
for e in posts.find({'active_search_score': 1}):    
    d = xgb.DMatrix([e['profile_embedding']])
    posts.update(
        { u'_id': e[u'_id'] },
        { '$set': { 'active_search_score': str(bst_loaded.predict(d)[0]) } }
    )

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """
